In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
data_path = '/content/drive/MyDrive/Movies Recommendation/Dataset/ml-32m/'

In [ ]:
movies = pd.read_csv(data_path + 'movies.csv', )       # movieId, title, genres
ratings = pd.read_csv(data_path + 'ratings.csv')    # userId, movieId, rating, timestamp
tags = pd.read_csv(data_path + 'tags.csv')           # userId, movieId, tag, timestamp
links = pd.read_csv(data_path + 'links.csv')         # movieId, imdbId, tmdbId

In [ ]:
tags = tags. sort_values(by=['userId', 'movieId', 'timestamp'], ascending= True)
ratings = ratings. sort_values(by=['userId', 'movieId', 'timestamp'], ascending = True)

In [ ]:
print(movies.count(), f"\n")
print(ratings.count(), f"\n")
print(tags.count(), f"\n")
print(links.count(), f"\n")

movieId    87585
title      87585
genres     87585
dtype: int64 

userId       32000204
movieId      32000204
rating       32000204
timestamp    32000204
dtype: int64 

userId       2000072
movieId      2000072
tag          2000055
timestamp    2000072
dtype: int64 

movieId    87585
imdbId     87585
tmdbId     87461
dtype: int64 



In [ ]:
ratings[ratings.duplicated(subset=['userId', 'movieId'])]

,userId,movieId,rating,timestamp


In [ ]:
tags[tags.duplicated(subset=['userId', 'movieId'])]

,userId,movieId,tag,timestamp
4,34,2174,weird,1249808102
8,58,7451,clique,1672551510
9,58,7451,coming of age,1672551502
10,58,7451,gossip,1672551527
11,58,7451,high school,1672551496
...,...,...,...,...
2000062,162279,90374,Hugh Dancy,1320817875
2000063,162279,90374,John Hawkes,1320817868
2000065,162279,90376,John C. Reilly,1325828372
2000066,162279,90376,Tilda Swinton,1325828338


In [ ]:
tags = tags.drop_duplicates(subset=['userId', 'movieId'], keep='last')

In [ ]:
tags[tags.duplicated(subset=['userId', 'movieId'])]

,userId,movieId,tag,timestamp


In [ ]:
tags = tags.drop(columns=['timestamp'])
ratings = ratings.drop(columns=['timestamp'])
links = links.drop(columns=['tmdbId'])

In [ ]:
df = pd.merge(movies, links, on='movieId', how='inner')
df = pd.merge(ratings, df, on='movieId', how='left')
df= pd.merge(df, tags, on=['movieId', 'userId'], how='inner')

In [ ]:
df.head()

,userId,movieId,rating,title,genres,imdbId,tag
0,22,26479,3.5,"Pirates of Penzance, The (1983)",Adventure|Comedy|Musical|Romance,86112,Kevin Kline
1,22,247150,3.0,Stowaway (2021),Drama|Sci-Fi,9203694,acrophobia
2,34,2174,4.0,Beetlejuice (1988),Comedy|Fantasy,94721,weird
3,34,8623,4.0,Roxanne (1987),Comedy|Romance,93886,Steve Martin
4,55,5766,4.0,Madman (1981),Horror,82696,the killls and the score


In [ ]:
df['year'] = df['title'].str.extract(r'\((\d{4})\)').astype('Int64')
df['title'] = df['title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip()

In [ ]:
df.isnull().sum()

,0
userId,0
movieId,0
rating,0
title,0
genres,0
imdbId,0
tag,0
year,1005


In [ ]:
df = df.dropna(subset=['year'])

In [ ]:
df.isnull().sum()

,0
userId,0
movieId,0
rating,0
title,0
genres,0
imdbId,0
tag,0
year,0


In [ ]:
df = df[df['genres'] != '(no genres listed)']

In [ ]:
df.count()

,0
userId,259450
movieId,259450
rating,259450
title,259450
genres,259450
tag,259450
year,259450
movielens_url,259450
imdb_url,259450


In [ ]:
df['movielens_url'] = 'https://movielens.org/movies/' + df['movieId'].astype(str)
df['imdb_url'] = 'http://www.imdb.com/title/tt' + df['imdbId'].astype(str).str.zfill(7)
df = df.drop(columns=['imdbId'])

In [ ]:
df.head()

,userId,movieId,rating,title,genres,tag,year,movielens_url,imdb_url
0,22,26479,3.5,"Pirates of Penzance, The",Adventure|Comedy|Musical|Romance,Kevin Kline,1983,https://movielens.org/movies/26479,http://www.imdb.com/title/tt0086112
1,22,247150,3.0,Stowaway,Drama|Sci-Fi,acrophobia,2021,https://movielens.org/movies/247150,http://www.imdb.com/title/tt9203694
2,34,2174,4.0,Beetlejuice,Comedy|Fantasy,weird,1988,https://movielens.org/movies/2174,http://www.imdb.com/title/tt0094721
3,34,8623,4.0,Roxanne,Comedy|Romance,Steve Martin,1987,https://movielens.org/movies/8623,http://www.imdb.com/title/tt0093886
4,55,5766,4.0,Madman,Horror,the killls and the score,1981,https://movielens.org/movies/5766,http://www.imdb.com/title/tt0082696


In [ ]:
df['movieId'].nunique()

26849

In [ ]:
output_path = '/content/drive/MyDrive/Movies Recommendation/Dataset/ml-32m/final_dataset.csv'
df.to_csv(output_path, index=False)